# Code Review Assistant - Streaming

Demonstrates streaming with code review tools.

In [3]:
!pip install --quiet -U langgraph langchain-anthropic


[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [4]:
from dotenv import load_dotenv
load_dotenv('studio/.env')

True

In [5]:
from langchain_anthropic import ChatAnthropic
from langchain_core.messages import SystemMessage, HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import MessagesState, StateGraph, START
from langgraph.prebuilt import tools_condition, ToolNode

# Code review tools
def check_syntax(code: str) -> str:
    """Check code for syntax errors."""
    if "syntax" in code.lower():
        return "Syntax error found: Missing semicolon"
    return "No syntax errors detected"

def check_style(code: str) -> str:
    """Check code for style compliance."""
    if len(code) < 10:
        return "Code too short for style analysis"
    return "Style check passed: Code follows PEP8 guidelines"

def suggest_improvements(code: str) -> str:
    """Suggest code improvements."""
    return "Consider adding:\n- Type hints\n- Docstrings\n- Better variable names"

tools = [check_syntax, check_style, suggest_improvements]
llm = ChatAnthropic(model="claude-3-5-haiku-20241022")
llm_with_tools = llm.bind_tools(tools)
sys_msg = SystemMessage(content="You are a code review assistant analyzing code quality.")

In [6]:
# Build graph
def assistant(state: MessagesState):
    return {"messages": [llm_with_tools.invoke([sys_msg] + state["messages"])]}

builder = StateGraph(MessagesState)
builder.add_node("assistant", assistant)
builder.add_node("tools", ToolNode(tools))
builder.add_edge(START, "assistant")
builder.add_conditional_edges("assistant", tools_condition)
builder.add_edge("tools", "assistant")

memory = MemorySaver()
graph = builder.compile(checkpointer=memory)

In [7]:
# Test: Review Python code
config = {"configurable": {"thread_id": 1}}
for event in graph.stream(
    {"messages": [HumanMessage(content="Review this code: def calculate(x): return x*2")]},
    config,
    stream_mode="values"
):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

Review this code: def calculate(x): return x*2
================================== Ai Message ==================================

[{'text': "I'll help you review this code by checking its syntax, style, and potentially suggesting improvements.\n\nFirst, I'll check the syntax:", 'type': 'text'}, {'id': 'toolu_01YHEWcK6ftGzjGj4C66brQ1', 'input': {'code': 'def calculate(x): return x*2'}, 'name': 'check_syntax', 'type': 'tool_use'}]
Tool Calls:
  check_syntax (toolu_01YHEWcK6ftGzjGj4C66brQ1)
 Call ID: toolu_01YHEWcK6ftGzjGj4C66brQ1
  Args:
    code: def calculate(x): return x*2
================================= Tool Message =================================
Name: check_syntax

No syntax errors detected
================================== Ai Message ==================================

[{'text': "Now, I'll check the style:", 'type': 'text'}, {'id': 'toolu_01HgSrAq65WaUAmrF3agosT5', 'input': {'code': 'def calculat